# Gathering and Adding Holidays 

In [19]:
import pandas as pd
import numpy as np
import datetime

Reading the data

In [20]:
dailytranswithweather = pd.read_csv('venue_daily_weather.csv')

Fetching the holidays happening in the USA and Canada

In [21]:
import requests

URL = "https://date.nager.at/api/v3/PublicHolidays/2024/"
canadaurl = URL + "CA"
canadaholidays = requests.get(url = canadaurl)

usaurl = URL + "US"
usaholidays = requests.get(url = usaurl)

canadadata = canadaholidays.json()
usadata = usaholidays.json()
holidaydata = canadadata + usadata
holidaylist = pd.DataFrame.from_dict(holidaydata)

Keeping only the national holidays

In [22]:

holidaylist = holidaylist[holidaylist['global'] == True]
holidaylist['holiday'] = 1
holidaylist.to_csv('Holidays_CanUS.csv', index=False) 


Adding holidays and weekends to the dataset - 1 if it was, 0 otherwise

In [24]:
mergedweatherwithholidays = pd.merge(dailytranswithweather, 
                                     holidaylist,  
                                     left_on = ['business_date', 'country'],  
                                     right_on = ['date', 'countryCode'],
                                     how ='left') 

mergedweatherwithholidays = mergedweatherwithholidays.drop(columns=['date_y', 'localName','name','fixed','global','counties','launchYear','types', 'countryCode'])

mergedweatherwithholidays['weekend'] = 0
for index,row in mergedweatherwithholidays.iterrows():
    # adding weekend
    date = datetime.datetime.strptime(row['business_date'], '%Y-%m-%d').date()
    if (date.isoweekday() > 5):
        mergedweatherwithholidays.at[index, 'weekend'] = 1

mergedweatherwithholidays['holiday'] = mergedweatherwithholidays['holiday'].fillna(0)
mergedweatherwithholidays.to_csv('WeatherandHolidays.csv', index=False)